In [27]:
'''
将产成品仓库的存货编码转为销售部的存货编码
'''

import pandas as pd
import numpy as np
import re
import openpyxl
import easygui
import os


#仓库销售编码字典
fname0 =  r"F:\a00nutstore\008\zw08\用友报价\仓库销售编码字典.xlsx"
df0 = pd.read_excel(fname0)
dic_ck_xs = dict(zip(df0['仓库货号'],df0['销售货号']))
dic_xs_ck = {v:k for k,v in dic_ck_xs.items()}

#产成品仓库库存
def chuliChangku(fname,sheet_name):
    df = pd.read_excel(fname,sheet_name = sheet_name)
    #删除第一列的空行
    df = df[~df['类别'].isna()]
    df = df[~df['类别'].str.contains('小计|累计|合   计')]
    df = df[~(abs(df['本日数量'] - 0) < 0.00001)]
    return df


In [29]:
fname = r"F:\a00nutstore\008\zw08\用友报价\7.23成品数据 的副本.xls"

sheet_name  =  0
df = chuliChangku(fname,sheet_name)

def addXiaoshouhuohao(df,dic_ck_xs):
    df_ck = df.assign(销售货号 = df['货号'].map(dic_ck_xs))
    # gp = df_ck.groupby('货号')
    gp = df_ck.groupby('销售货号')
    df_ck = pd.DataFrame(gp.sum())
    df_ck = df_ck[['货号','本日数量']]
    df_ck = df_ck.reset_index()
    return df_ck
df_ck = addXiaoshouhuohao(df,dic_ck_xs)
df_ck = df_ck[['本日数量','销售货号','货号']]
df_ck = df_ck.rename(columns = {'货号':'仓库货号'})
df_ck.to_excel('仓库库存.xlsx',index = False)

# 销售编码
def xiaoshou(fname_xs):
    df_xs = pd.read_excel(fname_xs)
    df_xs = df_xs[~df_xs['类别'].isna()]
    return df_xs





C:\Users\redda\AppData\Local\Temp\ipykernel_21548\811897962.py:17: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_ck.to_excel('仓库库存.xlsx',index = False)


In [30]:
fname_xs = r"F:\a00nutstore\008\zw08\用友报价\销售仓库价格库存1.xlsx"
df_xs = xiaoshou(fname_xs)
xs_lst = [
    '类别',
    '货号',
    '品名',
    '销售部存货大类编码',
    '含量',
    '汉办',
    '汉口北电商',
    '外地',
]
df_xs = df_xs[xs_lst]


def weiyihuXiaoshouhuohao(df_xs, df_ck, dic_xs_ck):
    merge = pd.merge(df_xs, df_ck, left_on='货号', right_on='销售货号', how='right')
    merge['本日数量本'] = merge['本日数量']
    merge['本日数量件'] = merge['本日数量'] / merge['含量']
    del merge['本日数量']
    del merge['销售货号']
    gp = merge.groupby('货号')
    dic_agg = {'含量': 'mean', '汉办': 'mean', '汉口北电商': 'mean', '外地': 'mean', '本日数量本': 'sum'}
    right = gp.agg(dic_agg)
    right = right.reset_index()
    left = df_xs[[
        '类别',
        '货号',
        '品名',
        '销售部存货大类编码'
    ]]
    mg = pd.merge(left, right, left_on='货号', right_on='货号',how = 'right')
    mg['本日数量件'] = mg['本日数量本'] / mg['含量']
    mg = mg.assign(仓库货号=mg['货号'].map(dic_xs_ck))
    return mg


result_beforeNew = weiyihuXiaoshouhuohao(df_xs, df_ck, dic_xs_ck)
result_beforeNew.to_excel('销售仓库价格库存0723.xlsx',index = False)

C:\Users\redda\AppData\Local\Temp\ipykernel_21548\3015964780.py:39: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  result_beforeNew.to_excel('销售仓库价格库存0723.xlsx',index = False)


In [48]:
#7月16-7月25日新系统累计出库
lst = [
'仓库编码',
    '仓库',   
    '存货分类 (1级)',
 '存货分类 (2级)',
 '存货分类 (3级)',
 '存货分类 (4级)',
 '存货分类 (5级)',
 '存货编码',
 '存货',
 '存货代码',
  '数量(主单位)',
 '平均单价',
 '金额',
 '数量(辅单位)',
 '计量单位组合',
 '数量(主单位).1',
 '平均单价.1',
 '金额.1',
 '数量(辅单位).1',
 '计量单位组合.1',
 '数量(主单位).2',
 '平均单价.2',
 '金额.2',
 '数量(辅单位).2',
 '计量单位组合.2',
 '数量(主单位).3',
 '平均单价.3',
 '金额.3',
 '辅数量',
 '计量单位组合.3']
lst2 = [
    '仓库编码',
    '仓库',  
'class01',
 'class02',
 'class03',
 'class04',
 'class05',
 'code',
 'stock',
 'content',
  'begin_ben',
 '平均单价',
 '金额',
 'begin_jian',
 '计量单位组合',
 'ruku_ben',
 '平均单价.1',
 '金额.1',
 'ruku_jian',
 '计量单位组合.1',
 'chuku_ben',
 '平均单价.2',
 '金额.2',
 'chuku_jian',
 '计量单位组合.2',
 'end_ben',
 '平均单价.3',
 '金额.3',
 'end_jian',
 '计量单位组合.3']

lst3 =  [
 'class02',
 'class05',
 'code',
 'stock',
 'content',
 'begin_jian',
 'ruku_jian',
    'chuku_ben',
 'chuku_jian',
 'end_jian',
]




def chuli(fname):
    df =  pd.read_excel(fname,skiprows = 7)
    df = df.iloc[:,1:]
    df.columns = lst
    df.columns = lst2
    df['content'] = df['end_ben']/df['end_jian']
    df = df[df['class01'] != '制表人:']
    df = df[df['class01'] != '合计：']
    df = df[df['class01'].notnull()]
    df = df[lst3]
    return df

fname = r"F:\a00nutstore\008\zw08\用友报价\产成品库存\收发存汇总表20240723.xlsx"
chuku = chuli(fname)

chuku_new = chuku[['code','begin_jian','ruku_jian','chuku_ben','chuku_jian']]
chuku_new = chuku_new.groupby('code').sum()
chuku_new = chuku_new.reset_index()
chuku_new

,code,begin_jian,ruku_jian,chuku_ben,chuku_jian
0,1101,37.02,0.04,1.0,0.02
1,1103,15.02,0.20,6.0,0.10
2,1105,15.00,0.00,0.0,0.00
3,1106,118.00,0.00,120.0,2.00
4,1107,19.00,0.00,0.0,0.00
...,...,...,...,...,...
1543,瑞典A460g线条,56.00,0.00,0.0,0.00
1544,瑞典A470g线条,1.00,0.00,0.0,0.00
1545,美国A6铁订本(枫叶)*,9.00,0.00,0.0,0.00
1546,美国A6铁订本*,7.00,0.00,0.0,0.00


In [41]:
chuku_new.to_excel('测试期间出库.xlsx',index = False)

C:\Users\redda\AppData\Local\Temp\ipykernel_21548\3083325343.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  chuku_new.to_excel('测试期间出库.xlsx',index = False)


In [62]:
#期初本、期初件
fname0 = r"F:\a00nutstore\008\zw08\用友报价\销售部001库存20240716正.xlsx"
df0 = pd.read_excel(fname0)
df0 = df0[['货号','本日数量本','本日数量件']]
df0 = df0.rename(columns = {'本日数量本':'qichu_ben','本日数量件':'qichu_jian'})
df0

,货号,qichu_ben,qichu_jian
0,1101,2220,37.0
1,1103,900,15.0
2,1105,900,15.0
3,1106,7080,118.0
4,1107,1140,19.0
...,...,...,...
1529,瑞典A460g方格,720,18.0
1530,瑞典A460g线条,2240,56.0
1531,瑞典A470g线条,40,1.0
1532,商品卡,113400,21.0


In [65]:
result0 = pd.merge(result_beforeNew,chuku_new,left_on = '货号',right_on = 'code',how = 'left')
result1 = pd.merge(result0,df0,on = '货号',how = 'left')
result1

,类别,货号,品名,销售部存货大类编码,含量,汉办,汉口北电商,外地,本日数量本,本日数量件,仓库货号,code,begin_jian,ruku_jian,chuku_ben,chuku_jian,qichu_ben,qichu_jian
0,普通账芯,1101,16K总分类账,01010201,60.0,5.5200,5.5200,5.5200,2220,37.0,1101,1101,37.02,0.04,1.0,0.02,2220.0,37.0
1,普通账芯,1103,16K银行存款日记账,01010201,60.0,5.5200,5.5200,5.5200,900,15.0,1103,1103,15.02,0.20,6.0,0.10,900.0,15.0
2,普通账芯,1105,16K现金日记账,01010201,60.0,5.5200,5.5200,5.5200,900,15.0,1105,1105,15.00,0.00,0.0,0.00,900.0,15.0
3,普通账芯,1106,16K三栏式明细分类账,01010201,60.0,5.5200,5.5200,5.5200,7020,117.0,1106,1106,118.00,0.00,120.0,2.00,7080.0,118.0
4,普通账芯,1107,16K固定资产明细分类账,01010201,60.0,5.5200,5.5200,5.5200,1140,19.0,1107,1107,19.00,0.00,0.0,0.00,1140.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552,外贸订制品,瑞典A460g线条,瑞典A460g线条,010214,40.0,0.0000,0.0000,0.0000,2240,56.0,瑞典A460g线条,瑞典A460g线条,56.00,0.00,0.0,0.00,2240.0,56.0
1553,外贸订制品,瑞典A470g线条,瑞典A470g线条,010214,40.0,0.0000,0.0000,0.0000,40,1.0,瑞典A470g线条,瑞典A470g线条,1.00,0.00,0.0,0.00,40.0,1.0
1554,外贸订制品,美国A6铁订本(枫叶)*,美国A6铁订本(枫叶)*,010214,36.0,0.0000,0.0000,0.0000,324,9.0,美国A6铁订本*,美国A6铁订本(枫叶)*,9.00,0.00,0.0,0.00,324.0,9.0
1555,外贸订制品,美国A6铁订本*,美国A6铁订本(枫叶)*,010214,36.0,0.0000,0.0000,0.0000,252,7.0,美国A6铁订本(枫叶)*,美国A6铁订本*,7.00,0.00,0.0,0.00,252.0,7.0


In [66]:
result1['期末件'] = result1['本日数量件'] - result1['chuku_jian']
result1['期末本'] = result1['本日数量本'] - result1['chuku_ben']
result1['变动件'] = result1['期末件'] - result1['qichu_jian']
result1['变动本'] = result1['期末本'] - result1['qichu_ben']

In [67]:
result1.to_excel('产成品变动.xlsx',index = False)


C:\Users\redda\AppData\Local\Temp\ipykernel_21548\1888454705.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  result1.to_excel('产成品变动.xlsx',index = False)
